# Peak matching for optimizing clustering and classification of XENONnT

Angevaare, Joran <j.angevaare@nikhef.nl> 

2021-04-22

## This notebook
In order to update clustering and classification parameters, this notebook investigates two properties of a simulated set of data:
 - `bias`, the area of the reconstructed peak / the area of the simulated peak
 - `arbitrary_acceptance`, slightly modified acceptance definition in order to allow penalty factors for bad reconstruction outcomes. The acceptance of the peaks is defined and explained in the [documentation](https://pema.readthedocs.io/en/latest/)
 
Using a grid scan of the relevant clustering and classification parameters, an optimum configuration can be extracted. This improved, changed configuration is finally compared to the current "default" reconstruction/

## Prerequisites 
#### Software
The following packages are needed
 - strax
 - straxen
 - wfsim
 - pema

#### Environment
This notebook can either be run on a local machine but is best run on the dali server

#### User input
The user needs to change few things, as the notebook is designed to work out of the box. Only few places (explicitly marked <span style="color:red">**user input required**</span> need changing).

**Start up ``strax(en) + wfsim + pema`` + load tools**

In [ ]:
import pema
import os

In [ ]:
if not os.path.exists('init.py'):
    init = os.path.join(pema.__path__[0], '..', 'bin', "pema_init.py")
    !ln -s $init init.py
%run init.py

# <span style="color:red">**User input required**</span> 
## Initialize the wavefrom simulator with your instructions

Initialize either of
 - [Low E](setup_lowe.py), energies between 0-50/250 PE (S1/S2). Follow this link to [edit](/edit/pema/notebooks/setup_lowe.py).
 - [High E](setup_highe.py), energies between 0-1e5 PE (S1 and S2). Follow this link to [edit](/edit/pema/notebooks/setup_highe.py).
 - [Kr](setup_kr.py), Kr-double scatters only (S1 and S2). Follow this link to [edit](/edit/pema/notebooks/setup_kr.py).
 
Each file contains the paths to a few folders and the setting up of a notebook on the dali computing cluster. Please change the following:
 - Folders (`base_dir, data_name, fig_dir, data_dir, raw_data_dir`)
 - Notebook starting requirements (`environ_init`)
 - The run_list of runs to simulated needed for gains etc. (`run_list`)
 - The instructions of the simulation itself such as #events, energies etc. (`instructions`)

In [ ]:
%run setup_lowe.py

### Initialize the context

In [ ]:
testing_config = {}

def get_context(skip=None):
    """
    Get a context and set all of the entries in the testing_config to the 
    context config IF they are not in 'skip' 
    """
    # Directories are set in the "%run setup_..." above
    st = pema.pema_context(base_dir=base_dir,
                           config_update=config_update,
                           raw_dir=raw_data_dir,
                           data_dir=data_dir)

    # Update the context config with all of the parameters that are not skipped
    for k, v in testing_config.items():
        if skip is not None and k in skip:
            print(f'skip {k}')
        else:
            st.set_config({k:v})

    # Don't allow lazy mode for any config
    st.set_context_config(
        {'allow_shm': True, 'allow_lazy': False,
         'timeout': 300, 'max_messages': 10,
        })
    return st

st = get_context()

### Simulate raw records
Below, we are going to simulate a number of runs, each run is processed in a different job on dali.

In [ ]:
# keep track of the running jobs in a list
job_registry=[]

for r in run_list:
    print(f'Starting a job to simulate data for run {r}')
    job = pema.ProcessRun(st, 
                          run_id=r, 
                          target=('records', 'peaklets'), 
                          config={})
    job_registry.append(job)
    cmd, job_name = job.make_cmd()
    
    # Alternatively look at job.exec_local
    job.exec_dali(cmd, job_name, environ_init, mem=30_000, max_hours='08:00:00')

In [ ]:
# We can check how the jobs are doing by checking their logs
for j in job_registry:
    log=j.log_file
    !tail -10 $log

Now, we simply have to wait for the runs to finish. We can see if all the data is stored by generating a table of stored items as below:

In [ ]:
script_writer = pema.ProcessRun(
    st,
    run_list,
    ('raw_records', 'records', 'peaklets', 'peaks_matched'))
script_writer.all_stored()

## Ready to go
Great, we are ready to go, we can do two things:
 1. We can first try to find an optimal parameter set using the sections below
 2. If we already know what parameter set we are going to need, we might just as well continue straight to:
 
 [**The validation section at the bottom of the notebook**](#Select-custom-config)

# Make configs for the parameters to change
Let's loop over the options we want to scan. We will write those to a dict and work from there.

<span style="color:red">**User input may be required**</span> depending on if one wants to look at some other parameters. Below, we will simply look at close to **all** of the clustering and classification parameters but this many naturally be overkill. If one simply wants to look at a single parameter, this can be greatly simplified.

Let's load all the parameters related to peaks. Those are the ones we can optimize. We want to know their current value so that we can look around to see how things change.

In [ ]:
opts = st.show_config('peaklets')
# Optionally, one may look parameters applying to peaklets by commenting out line below
# opts = opts[opts['applies_to'] == ('peaklets', 'lone_hits')]
opts = opts.to_records()

Parse the current config, if there is an `current` option overwriting the `default`, do so here too.

In [ ]:
options = opts['option']
values = opts['default']
current = opts['current']
mask = current != '<OMITTED>'
values[mask] = current[mask]
res = {k: v for (k, v) in zip(options, values)}

OK, let's hard code for a second which parameters we are interested in. Most of the other parameters are booleans etc. which are not suitable for this parameter scan.

In [ ]:
keep_keys = [
 'peaklet_gap_threshold',
 'peak_left_extension',
 'peak_right_extension',
 'peak_split_filter_wing_width',
 'peak_split_min_area',
 'peak_split_iterations',
 'tight_coincidence_window_left',
 'tight_coincidence_window_right',
 's1_max_rise_time',
 's1_max_rise_time_post100',
 's2_merge_max_area',
 's2_merge_max_gap',
 's2_merge_max_duration'
]
res = {k: v for (k,v )in res.items() if k in keep_keys}
summary_config = res.copy()
res

In [ ]:
def validate_conf(conf):
    """
    Check right and left extension to be allowable. That is, we obey:
    gap_threshold > left_extension + right_extension
    """
    gap_threshold = conf.get('peaklet_gap_threshold', 
                             res.get('peaklet_gap_threshold', 700))
    left_extension = conf.get('peak_left_extension', 
                              res.get('peak_left_extension', 30))
    right_extension = conf.get('peak_right_extension', 
                               res.get('peak_right_extension', 350))
    return gap_threshold > left_extension + right_extension

#### let's make a list of configs we want to validate
To this end, we loop over all the configs selected above and vary the value of each parameter in a simple for loop. The goal is to vary around the current value to allow a large parameter scan.

In [ ]:
conf_tot = [{}]
i=0
keys_seen = []
_st = st.new_context()

# Loop over teh configs and add new configs to the config list
for k, v in summary_config.items():
    if k == 'peak_split_gof_threshold': 
        continue
    _type = type(v)
    
    for factor in [2.5, 2, 1.5, 1.25, 1.1, 1,
                   1/1.1, 1/1.25, 1/1.5, 1/2, 1/2.5    ]:
        value = _type(v*factor)
        conf = {k: value}
        if not validate_conf(conf):
            print(f'skip {conf}')
            continue
        # Let's check that this config is not already seen, otherwise just
        # continue
        _st.set_config(conf)
        ev_key = _st.key_for('0', 'events')

        if str(ev_key) in keys_seen and factor != 1:
            continue
        else:
            # New config! Let's store it
            conf_tot.append(conf)
            keys_seen.append(str(ev_key))

Brute force method for varying the `peak_split_gof_threshold`. This parameter (used in natural break splitting) is a `2 *4` tuple and therefor, many things can be optimized. 

The loop below is extremely brute force but does the trick. Keep in mind that looping over 8 parameters very very quickly becomes a huge set.

In [ ]:
# factors = [1.5, 
#            1.25, 1, 0.8, 
#            0.5
# ]
# few_factors = [1.25, 1, 0.8]
# for fa in factors:
#     for fb in factors:
#         for fc in few_factors:
#             for fd in few_factors:
#                 for fx in few_factors:
#                     for fy in factors:
#                         for fz in few_factors:
#                             for fw in few_factors:
#                                 if not fa+fb+fc+fd+fx+fy+fz+fw in 7 + np.array(factors):
#                                     continue
#                                 if fa >1 or fb>1 or fc>1 or fd>1:
#                                     continue
# #                                 if fc!=1 or fw != 1:
# #                                     continue
#                                 value = (None, 
#                                          ((0.5*fx, 1*fa), (6.0*fy, 0.4*fb)), 
#                                          ((2.*fz, 1*fc), (4.5*fw, 0.4*fd)))
#                                 conf = {'peak_split_gof_threshold': value}
#                                 _st.set_config(conf)
#                                 ev_key = _st.key_for('0', 'events')

#                                 if str(ev_key) in keys_seen:
#                                     continue
#                                 else:
#                                     conf_tot.append(conf)
#                                     keys_seen.append(str(ev_key))

In [ ]:
confs = tuple(conf_tot.copy())
print(f'Working with {len(conf_tot)} configs, see below\n{confs}')

# Submit jobs for computing the results of configs
Now, the fun bit starts, let's run each of the configs in a seperate job. 

Make sure that the raw-records simulations have finished first!

In [ ]:
# only use runs where we have the records stored
selected_runs = [r for r in run_list if st.is_stored(r, 'records')]
print(f'Doing runs:\n{selected_runs}\n{len(selected_runs)/len(run_list)*100:.1f}%')
all_runs = len(selected_runs) == len(run_list)

Iterate over the configs (this allows us to stop the submit-cell and restart)

In [ ]:
iter_confs = iter(confs)
job_registry = []

In [ ]:
# Parameters needed for the job submission
target = ('raw_records', 'records',  
          'peaklets', 'peak_basics', 'match_acceptance_extended',)
RAM = 6_000
queue_max = 110
check_que_after = 10
part = 'xenon1t'
max_hours='01:00:00'

# start the jobs one by one
for i, conf in enumerate(tqdm(
    iter_confs, 
    total = len(confs) - len(job_registry ), 
    desc='configs')):
    
    job = pema.ProcessRun(st, run_id=selected_runs, target=target, config=conf)
    cmd, job_name = job.make_cmd()

    job_registry.append(job)
    # Submit the job if some of the targets is not stored yet.
    if not job.all_stored(return_bool=True):
        # one could clean up some data if some run keeps failing to process correctly
        # job.purge_below('peak_basics')
        # job.purge_below('match_acceptance_extended')
        job.exec_dali(cmd, job_name, environ_init, mem=RAM, 
                      max_hours=max_hours, partition=part)

    # Check if we are not getting too many jobs
    if i % check_que_after:
        q = !squeue -u `echo $USER`
        while len(q)> queue_max:
            q = !squeue -u `echo $USER`
            print(f'waiting 10s, queue is full. {len(q)}')
            time.sleep(10)

Wait for the runs to finish before continueing

In [ ]:
while True:
    nrun = !squeue -u $USER  | wc -l
    nrun = int(nrun[0])
    print(nrun)
    # if we have less than 5 jobs running, we are probably done, otherwise, wait
    if nrun < 5:
        break
    time.sleep(10)

Check if everything is stored (can take a *very* long time)

In [ ]:
# pd.set_option('display.max_rows', 500)
pd.concat([j.all_stored(show_key=True) for j in job_registry])

# Check the outcomes of the simulated data

In [ ]:
def compute_acceptance(data):
    """
    Simple function to calculate the mean acceptance fraction of an entire 
    dataset (incl. the binom. error)
    """
    total = len(data)
    found = np.sum(data['acceptance_fraction'])
    return found/total, pema.binom_interval(found, total)

In [ ]:
def compute_bias(data):
    """
    Simple function to calculate the mean bias of an entire dataset (incl. std)
    """
    total = len(data)
    sub_sel = data['rec_bias'] > 0 
    return np.mean(data['rec_bias'][sub_sel]), np.std(data['rec_bias'][sub_sel])

Summarize the results

We loop over each of the jobs, if they are finished, let's check the mean acceptance and mean bias of that configuration

In [ ]:
res = defaultdict(list)

for i, job in enumerate(tqdm(job_registry)):
    config = job.config
    if config in res['config']:
        continue
    elif not job.all_stored(return_bool=True):
        print(f'skip {job}')
        continue
    
    data = job.st.get_array(selected_runs, 
                            'match_acceptance_extended', 
                            progress_bar=False)

    # Parse the average mean and acceptance and store in our "res" list
    res['number'].append(i)
    res['config'].append(config)
    res['config_type'].append(list(config.keys()))
    for si in range(1,3):
        sel = data['type'] == si
        acceptance, (low, high) = compute_acceptance(data[sel])
        res[f's{si}_acc'].append(acceptance)
        res[f's{si}_acc_err'].append([acceptance-low, high-acceptance])

        bias_mean, bias_err = compute_bias(data[sel])
        res[f's{si}_bias'].append(bias_mean)
        res[f's{si}_bias_err'].append(bias_err)

The results look like this

In [ ]:
df = pd.DataFrame(res)
df

## Interpret parameter scan
For each of the configs, we are going to grep the results from our dataframe above to see how the accepance/bias was affected by this parameter change.

In [ ]:
# Common parameters for plot: S1/S2 colors and errorbar options
colors = ('#1f77b4', '#ff7f0e')
plot_kwargs = dict(markersize= 5,  ls='none', capsize=3, marker='o')

for config_type in np.unique(df['config_type'].values):
    if not config_type:
        continue
    print(config_type)
    
    # Select the configurations from the results dataframe
    mask = np.array([c == config_type for c in  df['config_type']])
    
    # Plot
    fig = plt.figure(figsize=(1*np.sum(mask),6))
    plt.title(f'S1/S2 acceptance - {config_type}')
    for axi, si in enumerate([1, 2]):
        if axi ==1:
            plt.sca(plt.gca().twinx())
            plt.xticks()
        plt.errorbar(df[mask]['number'], 
                     df[mask][f's{si}_acc'], 
                     yerr=np.array([e for e in df[mask][f's{si}_acc_err']]).T,
                     label = f'S{si} acceptance',
                     c = colors[axi],
                     **plot_kwargs,
                    )
        plt.axhline(df[f's{si}_acc'][0], ls = '--', c = colors[axi], label=f'default S{si} acceptance')
        plt.ylabel(f'S{si} acceptance')
        plt.gca().yaxis.label.set_color(colors[axi])
        plt.gca().tick_params(axis='y', colors=colors[axi])

        if axi==0:
            plt.xticks(df[mask]['number'], 
                       df[mask]['config'], 
                       rotation = 45, ha='right')

    fig.legend(loc=5)
    # Save the figure
    pema.save_canvas(f'{config_type}_update_lowe_scan', 
                     save_dir=os.path.join(fig_dir, 'update_scan'))
    plt.show()

Do the same for bias

In [ ]:
for config_type in np.unique(df['config_type'].values):
    if not config_type:
        continue
    print(config_type)
    # Select the configurations from the results dataframe
    mask = np.array([c == config_type for c in  df['config_type']])
    
    # Plot
    fig = plt.figure(figsize=(1*np.sum(mask),6))
    plt.title(f'S1/S2 acceptance - {config_type}')
    for axi, si in enumerate([1, 2]):
        if axi ==1:
            plt.sca(plt.gca().twinx())
            plt.xticks()
        plt.errorbar(df[mask]['number'], 
                     df[mask][f's{si}_bias'], 
                     yerr=df[mask][f's{si}_bias_err'],
                     label = f'S{si} bias',
                     c = colors[axi],
                     **plot_kwargs,
                    )
        plt.axhline(df[f's{si}_bias'][0], ls = '--', c = colors[axi], label=f'default S{si} acceptance')
        plt.ylabel(f'S{si} bias')
        plt.gca().yaxis.label.set_color(colors[axi])
        plt.gca().tick_params(axis='y', colors=colors[axi])
        
        if axi==0:
            plt.xticks(df[mask]['number'], 
                       df[mask]['config'], 
                       rotation = 45, ha='right')
    fig.legend(loc=5)
    # Save the figure
    pema.save_canvas(f'{config_type}_update_lowe_bias_scan', save_dir=os.path.join(fig_dir, 'update_scan'))
    plt.show()

# Select custom config
## <span style="color:red">**User input required**</span> 
From the plots above, we can now extract a desired change to the default clustering and classification parameters.

An improved parameter:
 1. Reduces the bias (i.e. brings it closer to a value of `1.`)
 2. Increases the acceptance
 3. Works for S1/S2 and for lowE / HighE

Here, obviously, item `3.` is by far the most complex criterion to fullfil. Most parameters are quite optimized and have conflicting effects on e.g.g the S1- vs. S2-acceptance. As such, this is a step that needs to be done by a user since this weighing of interests is a complex task.

### Example
Below we are going to set the extracted config to
```python
summary_config = {
    's2_merge_max_duration': 40000,
    's2_merge_max_gap': 9000,
    'peaklet_gap_threshold': 1000
                 }
```
This config is not actually better than the current default settings but serves the purpose of being an example. 

Using this example, we make some detailed plots of how this config affects the bias, waveforms and acceptance. We keep the results of default (`st`) and custom (`st2`) outcomes side by side in two contexts.

In [ ]:
summary_config = {
    's2_merge_max_duration': 40000,
    's2_merge_max_gap': 9000,
    'peaklet_gap_threshold': 1000
                 }
st2 = st.new_context()
st2.set_config(summary_config)

In [ ]:
default_acceptence = st.get_array(selected_runs, 'match_acceptance_extended',progress_bar=False)
custom_acceptence = st2.get_array(selected_runs, 'match_acceptance_extended',progress_bar=False)

Acceptance comparison plots

In [ ]:
def si_acceptance(si, binedges, on_axis='n_photon', nbins=50):
    mask = default_acceptence['type'] == si
    pema.summary_plots.acceptance_plot(
        default_acceptence[mask], 
        on_axis, 
        binedges, 
        nbins=nbins, 
        plot_label=default_label,
    )
    mask = custom_acceptence['type'] == si
    pema.summary_plots.acceptance_plot(
        custom_acceptence[mask], 
        on_axis, 
        binedges, 
        nbins=nbins, 
        plot_label=custom_label,
    )
    plt.ylabel('Arb. Acceptance')
    plt.title(f"S{si} acceptance")
    plt.legend()

def acceptance_summary(si, on_axis, axis_label, nbins = 100, plot_range = (0, 200), save_name=''):
    f, axes = plt.subplots(3, 1, figsize=(10,12), sharex=True)
    max_photons = 35
    plt.sca(axes[0])
    sel = ((default_acceptence['type'] == si) 
           & (default_acceptence[on_axis] > plot_range[0])
           & (default_acceptence[on_axis] < plot_range[1])
          )
    pema.summary_plots.plot_peak_matching_histogram(default_acceptence[sel], on_axis, bin_edges = nbins)
    plt.text(0.05,0.95, 
             default_label,
             transform=plt.gca().transAxes,
             ha = 'left',
             va = 'top',
             bbox=dict(boxstyle="round", fc="w")
            )
    plt.legend(loc=(1.01,0))
    plt.xlim(*plot_range)
  
    plt.sca(axes[1])
    sel = ((custom_acceptence['type'] == si) 
           & (custom_acceptence[on_axis] > plot_range[0])
           & (custom_acceptence[on_axis] < plot_range[1])
          )
    print(f'cust {np.sum(sel)}')
    pema.summary_plots.plot_peak_matching_histogram(custom_acceptence[sel], on_axis, bin_edges = nbins)
    plt.text(0.05,0.95, 
             custom_label,
             transform=plt.gca().transAxes,
             ha = 'left',
             va = 'top',
             bbox=dict(boxstyle="round", fc="w")
            )
    plt.legend(loc=(1.01,0))
    plt.xlim(*plot_range)
    
    plt.sca(axes[2])
    mask = default_acceptence['type'] == si
    pema.summary_plots.acceptance_plot(default_acceptence[mask], on_axis, plot_range, nbins=nbins, 
                                       plot_label=default_label)
    mask = custom_acceptence['type'] == si

    pema.summary_plots.acceptance_plot(custom_acceptence[mask], on_axis, plot_range, nbins=nbins, 
                                       plot_label=custom_label)
    plt.legend(loc=(1.01,0))
    plt.ylabel('Arb. acceptance faction')
    plt.xlim(*plot_range)
    plt.xlabel(axis_label)
    plt.ylim(0,1)

    plt.subplots_adjust(hspace=0)
    plt.suptitle(f'S{si} Acceptance', y=0.9)
    pema.save_canvas(f'{si}_acceptance_detailed_{save_name}', save_dir=fig_dir)

In [ ]:
acceptance_summary(si = 1, 
                   on_axis = 'n_photon',
                   axis_label = 'N photons simulated', 
                   nbins = 100, 
                   plot_range = (0, 50),
                   save_name = 'tot_compare',)

acceptance_summary(si = 2, 
                   on_axis = 'n_photon',
                   axis_label = 'N photons simulated', 
                   nbins = 100, 
                   plot_range = (0, 250),
                  save_name = 'tot_compare')

acceptance_summary(si = 2, 
                   on_axis = 'z',
                   axis_label = 'z (simulated) [cm]', 
                   nbins = 75, 
                   plot_range = (-160, 10),
                   save_name = 'tot_compare')

reconstruction bias

In [ ]:
s1_max = default_acceptence['n_photon'][default_acceptence['type']==1].max()
s2_max = default_acceptence['n_photon'][default_acceptence['type']==2].max()

In [ ]:
pema.summary_plots.rec_diff(
    default_acceptence,
    custom_acceptence,
    s1_kwargs=dict(bins=[100, 100], range=[[0,s1_max], [0.6, 1.1]]),
    s2_kwargs=dict(bins=[100, 100], range=[[0,s2_max], [0.6, 1.1]])
        )

Compare wavefroms! This is very important to see where the improvements are coming from.

In [ ]:
pema.compare_outcomes(st, default_acceptence,                 
                      st2, custom_acceptence,
                      only_different=False,
                      plot_fuzz=3000,
                      fuzz=0,
                      max_peaks=10)